# List of websites and t&c patterns

## Loading a big website list

Here we manually listed a few websites, but the idea would be to launch a big csv to fill our Lake quickly :-)

In [1]:
#manually 

websites_try = [
'https://www.linkedin.com/',
'https://www.booking.com/index.html?aid=1743217',
'https://travelsites.com/expedia/',
'https://travelsites.com/tripadvisor/',
'https://www.momondo.in/?ispredir=true',
'https://www.ebookers.com/?AFFCID=EBOOKERS-UK.network.affiliatewindow.85386&awc=7904_1606029543_518c2af76500871f76cadec26f8eb055',
'https://book.priceline.com/?refid=8431&refclickid=d196c1c3cad4c6e474196ac6815f9652',
'https://www.trip.com/index?locale=en_us&allianceid=21934&SID=455408&ouid=198c18a52c8911eb811b00ce0a18050d',
'https://www.orbitz.com/',
'https://www.travelocity.com/',
'https://www.hotwire.com/',
'https://www.otel.com/',
'https://www.ebookers.com/?AFFCID=EBOOKERS-UK.network.affiliatewindow.85386&awc=7904_1606041576_a3c281bcb3eae0af35b22b076d1074ae',
'https://www.airbnb.co.in/?locale=en&_set_bev_on_new_domain=1606041642_YjI0OGQ3OWI5MDI2',
'https://in.lastminute.com/'
]

In [2]:
import pandas as pd
import csv

In [19]:
# csv loading
"""
with open('file.csv', newline='') as f:
    reader = csv.reader(f)
    websites_data = list(reader)

print(websites_data)"""

"\nwith open('file.csv', newline='') as f:\n    reader = csv.reader(f)\n    websites_data = list(reader)\n\nprint(websites_data)"

## determining patterns inside website 

The idea of this section is to list "all" the possible appelation of t&c on websites. The more we know how t&c are called on websites, the more we will be able to scrape it :-)

In [3]:

#defining a list of all possible names of t&c
tc_names = ['Terms and conditions','Terms of Use','Terms of use','Terms of Service', \
            'Terms of service','User Agreement','Terms','terms', 'Privacy & terms',\
            ' User Agreement ','Terms & conditions', 'Terms & Conditions', \
            'General conditions', 'Terms & Conditions', 'Terms and Conditions',\
            'Terms and conditions']

# idea for opti : rank the appelations in the list based on the popularity of it
# the more we find "Terms of Use" on websites, the more likely this appelation
# will be at the beginning of the list

# Creating our first scrapers

## The scrapers :-)

A section dedicated to the differents scrappers functions 

In [4]:
# importing necessary packages
from os import path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup 
import requests
import csv


### The t&c url scrapper

The scraper in this section has one mission : find the t&c page url on every website ! 

In [16]:
# Now we need to create a scraping function that will find the t&c url inside a website

def tcLinkExtractor(url,tc_names,time=5):
    # Chromedriver options
    options = Options()
    options.add_argument('headless') #avoir chromedriver opening
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument("--lang=en"); #set english as language
    
    # setting the driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    driver.implicitly_wait(time) #wait X sec to be sure the webpage is fully loaded
    source = driver.page_source #defines html code source as source var
        
    
    
    #creating an empty variable to return "" if no link if found
    tc_url = ""
    
    for tc_name in tc_names:
        try:
            tc_url = driver.find_element(By.XPATH,f"//*[text()[contains(., '{tc_name}')]]").get_attribute('href')
            break
        except:
            #print(f"'{tc_name}' not working for {url}")
            #nw_working_site = tc_name
            pass
    driver.quit()  
    
    if tc_url == "":
        print(f"{url} : t&c not matched ")
        return tc_url
    
    else:
        print(f"bingo for {url} :-) ")
        return tc_url

### the t&c page TEXT extractor

This scraper will extract the text from every webpage we feed him with

In [23]:
def tcTextScraper(url):
    #scrap all the text from a t&c webpage 
    
    # Chromedriver options
    options = Options()
    options.add_argument('headless') #avoir chromedriver opening
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    options.add_argument("--lang=en"); #set english as language
    
    # setting the driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    driver.implicitly_wait(5) #wait X sec to be sure the webpage is fully loaded
    source = driver.page_source
    driver.quit()

    # dumb scraping function that returns all text (from a t&c) webpage
    soup = BeautifulSoup(source, "html.parser")
    
    result = soup.text
    return result

#first try with an url directly aiming to terms and services of a website
#url_test = "https://www.surveymonkey.com/mp/legal/terms-of-use/"
#print(tcTextScraper(url_test))

## scrapper quick testing section

To try how (i?) updates work :) 
=> It will launch the tcLinkExtractor on a basic website list to see if everything works

In [14]:
# A first list of websites (to use to try scrapper)

websites = [
    "https://www.linkedin.com",
    "https://wikipedia.org",
    "https://www.producthunt.com",
    "https://stripe.com",
    "https://www.facebook.com"
]

In [15]:
tc_urls = []
for website in websites:
    tc_urls.append(tcLinkExtractor(website,tc_names,1))

tc_urls



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


bingo for https://www.linkedin.com :-) 




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


bingo for https://wikipedia.org :-) 


Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


bingo for https://www.producthunt.com :-) 




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


bingo for https://stripe.com :-) 
bingo for https://www.facebook.com :-) 


['https://www.linkedin.com/legal/user-agreement?trk=homepage-basic_footer-user-agreement',
 'https://meta.wikimedia.org/wiki/Terms_of_use',
 'https://www.producthunt.com/legal#terms',
 'https://stripe.com/en-fr/privacy',
 'https://www.facebook.com/policies?ref=pf']

# launching the scraper : extracting t&c urls

Here we try the scrapper in the real world to (try to) extract a maximum of t&c urls.

In order for the scrapper to work you must use the <b>tcLinkExtractor(data,tc_names,time) </b>function with those arguments :

- <b>data</b> = the website list you want to launch the scraper on
- <b>tc_names</b> = the list of t&c appelations to launch. Use "tc_names" by default
- <b>time</b> = the nb of seconds you want the scraper to wait each time he loads a website (defaut = 2 but it can be optimized)

In [6]:
tc_urls = []
for website in websites_try:
    tc_urls.append(tcLinkExtractor(website,tc_names,2))

tc_urls



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


https://www.linkedin.com/ : t&c not matched 




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


https://www.booking.com/index.html?aid=1743217 : t&c not matched 




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


...



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome


bingo for https://www.airbnb.co.in/?locale=en&_set_bev_on_new_domain=1606041642_YjI0OGQ3OWI5MDI2 :-) 


Driver [/Users/lejeunefrancois/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


https://in.lastminute.com/ : t&c not matched 


['',
 '',
 'https://travelsites.com/terms-and-conditions/',
 'https://travelsites.com/terms-and-conditions/',
 'https://www.momondo.in/terms-of-use',
 '',
 '',
 'https://pages.trip.com/service-guideline/terms-en-us.html',
 '',
 '',
 'https://www.hotwire.com/en/content/terms-use?cc=us',
 '',
 '',
 'https://www.airbnb.co.in/terms',
 '']

=> Once the scraper finished his job (all websites were parsed), you can run the following command to see the performance ( = for how many % of provided websites the scraper managed to find the t&c page)

In [7]:
# We compute the % of t&c urls
leng = len(tc_urls)
okies = leng - tc_urls.count("")

result = okies/leng
print(f"Our magnificient scrapper found {result*100} % of t&c pages ({okies} on {leng})")

Our magnificient scrapper found 40.0 % of t&c pages (6 on 15)
